In [5]:
! pip install pyspark
! pip install py4j


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
! pip install pandas


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
emp_df = spark.read.csv('/content/employees.csv', header=True, inferSchema=True)
emp_df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [ ]:
dept_df = spark.read.csv('/content/departments.csv', header=True, inferSchema=True)
dept_df.show()

+-------------+--------------------+----------+-----------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+--------------------+----------+-----------+
|           10|      Administration|       200|       1700|
|           20|           Marketing|       201|       1800|
|           30|          Purchasing|       114|       1700|
|           40|     Human Resources|       203|       2400|
|           50|            Shipping|       121|       1500|
|           60|                  IT|       103|       1400|
|           70|    Public Relations|       204|       2700|
|           80|               Sales|       145|       2500|
|           90|           Executive|       100|       1700|
|          100|             Finance|       108|       1700|
|          110|          Accounting|       205|       1700|
|          120|            Treasury|        - |       1700|
|          130|       Corporate Tax|        - |       1700|
|          140|  Control And Credit|    

In [ ]:
emp_df.write.json('/content/employees').mode('overwrite')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/employees already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
dept_df.write.json('/content/departments').mode('overwrite')

In [ ]:
emp_df.columns

['EMPLOYEE_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'SALARY',
 'COMMISSION_PCT',
 'MANAGER_ID',
 'DEPARTMENT_ID']

In [ ]:
dept_df.columns

['DEPARTMENT_ID', 'DEPARTMENT_NAME', 'MANAGER_ID', 'LOCATION_ID']

•	Perform a left  join between two DataFrames.

In [ ]:
emp_df.join(dept_df, emp_df.DEPARTMENT_ID == dept_df.DEPARTMENT_ID, 'left').show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID| DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|           50|        Shipping|       121|       1500|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|           50|        Shipping|       121|       1500|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |      

•	Use a window function to calculate the rank of employees based on their salaries within each department.

In [ ]:
from pyspark.sql.window import Window

In [ ]:
windowSpec = Window.partitionBy("DEPARTMENT_ID").orderBy(F.desc("SALARY"))

In [ ]:
ranked_df = emp_df.withColumn("rank", F.rank().over(windowSpec))

In [ ]:
ranked_df.orderBy("rank").show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+----+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|rank|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+----+
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|   1|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|   1|
|        114|       Den| Raphaely|DRAPHEAL|515.127.4561|07-DEC-02|    PU_MAN| 11000|            - |       100|           30|   1|
|        203|     Susan|   Mavris| SMAVRIS|515.123.7777|07-JUN-02|    HR_REP|  6500|            - |       101|           40|   1|
|        121|      Adam|    Fripp|  AFRIPP|650.123.2234|10-APR-05|    ST_MAN|  8200|      

•	Write a DataFrame to a Parquet file with partitioning by a specific column.